In [1]:
# importação das bibliotecas utilizadas
import json
from difflib import SequenceMatcher

import pandas as pd
import seaborn as sns

In [2]:
# Marcar como True para reproduzir as etapas mais lentas e verbosas do processo
VERBOSE = False

In [3]:
# carregando os filmes indicados ou vencedores do Oscar em um dataframe
# e examinando as características do dataframe
oscar_nominations = pd.read_csv('/kaggle/input/the-oscar-award/the_oscar_award.csv')

oscar_nominations.info()
oscar_nominations.describe()

In [4]:
# A única coluna com itens faltantes é a coluna film.
# Vamos explorar as linhas em que faltam esses dados para entender o porquê,
# e em quais categorias isso está ocorrendo
print(oscar_nominations[oscar_nominations['film'].isnull()]['category'].unique())
oscar_nominations[oscar_nominations['film'].isnull()].head()

In [5]:
# Alguns desses casos se tratam de prêmios honorários, humanitários e memoriais -
# ou seja, não relacionados a filmes específicos. Podemos desconsiderar essas linhas
rows_to_drop = oscar_nominations[
    (oscar_nominations['category'] == 'HONORARY AWARD') |
    (oscar_nominations['category'] == 'SPECIAL AWARD') |
    (oscar_nominations['category'] == 'IRVING G. THALBERG MEMORIAL AWARD') |
    (oscar_nominations['category'] == 'JEAN HERSHOLT HUMANITARIAN AWARD') |
    (oscar_nominations['category'] == 'SPECIAL ACHIEVEMENT AWARD')
].index

oscar_nominations = oscar_nominations.drop(rows_to_drop)

print(oscar_nominations[oscar_nominations['film'].isnull()]['category'].unique())
oscar_nominations[oscar_nominations['film'].isnull()].head()

In [6]:
# As linhas restantes com dados faltantes na coluna filme são de categorias
# que precisam necessariamente estar ligadas a um filme.

# Para duas dessas categorias, ligadas a filmes estrangeiros, o nome do filme parece constar
# na coluna 'name'

for row in oscar_nominations[
    (oscar_nominations['category'] == 'SPECIAL FOREIGN LANGUAGE FILM AWARD') |
    (oscar_nominations['category'] == 'HONORARY FOREIGN LANGUAGE FILM AWARD')
].iterrows():
    print(row[1][4])

# em todos esses casos, com exceção do último, o nome do filme consta na coluna 'name',
# seguido de hífen. Podemos aplicar a mesma regra para todos os casos, e transpor o nome do
# filme para a coluna film
oscar_nominations['film'] = oscar_nominations.apply(
    lambda x: x['name'].split('-')[0] if x['category'] in (
        'SPECIAL FOREIGN LANGUAGE FILM AWARD', 'HONORARY FOREIGN LANGUAGE FILM AWARD'
    )
    else x['film'],
    axis=1
)

In [7]:
# Com os dados dessas categorias corrigidos,
# é hora de checar os dados que ainda são faltantes
print(oscar_nominations[oscar_nominations['film'].isnull()]['category'].unique())
oscar_nominations[oscar_nominations['film'].isnull()].head()

In [8]:
# Outras linhas com dados faltantes na coluna 'film' são de obtenção difícil ou imprecisa:
# um assistente de direção, por exemplo, pode ter tido mais de um filme lançado num mesmo
# ano, o que faria a checagem dos dados restantes muito trabalhosa.
# Essas colunas serão ignoradas.
rows_to_drop = oscar_nominations[
    (
        (oscar_nominations['category'] == 'ENGINEERING EFFECTS') |
        (oscar_nominations['category'] == 'WRITING (Title Writing)') |
        (oscar_nominations['category'] == 'SOUND RECORDING') |
        (oscar_nominations['category'] == 'ASSISTANT DIRECTOR')
    ) & (oscar_nominations['film'].isnull())
].index

oscar_nominations = oscar_nominations.drop(rows_to_drop)

oscar_nominations.info()

In [9]:
# Um fato que pode ser observado analisando esses dados restantes é que algumas das
# categorias tiveram mudanças de nomes - como foi o caso na premiação de filme de língua
# estrangeira. Já outras categorias desapareceram da premiação ("melhor filme em preto e
# branco") ou foram incluídas.

# A categoria mais recente incluída na premiação é a de melhor filme de animação, criada em
# 2002. Por isso, iremos desconsiderar filmes anteriores a essa data.

rows_to_drop = oscar_nominations[(oscar_nominations['year_ceremony'] < 2002)].index
oscar_nominations = oscar_nominations.drop(rows_to_drop)

# Iremos desconsiderar ainda a premiação de edição de som (sound editing),
# que foi descontinuada em 2019, a única removida após o ano de 2002.
rows_to_drop = oscar_nominations[(oscar_nominations['category'] == 'SOUND EDITING')].index
oscar_nominations = oscar_nominations.drop(rows_to_drop)

list(oscar_nominations['category'].unique())

In [10]:
# Algumas dessas categorias também precisaram de consolidação, já que seus nomes
# aparecem de formas diferentes. Foram utilizados os nomes mais recentes.

category_renaming = {
    'FOREIGN LANGUAGE FILM': 'INTERNATIONAL FEATURE FILM',
    'ART DIRECTION': 'PRODUCTION DESIGN',
    'WRITING (Screenplay Based on Material Previously Produced or Published)': 'WRITING (Adapted Screenplay)',
    'WRITING (Screenplay Written Directly for the Screen)': 'WRITING (Original Screenplay)',
    'SOUND MIXING': 'SOUND',
    'MAKEUP': 'MAKEUP AND HAIRSTYLING',
}

oscar_nominations['category'] = oscar_nominations.apply(
    lambda x: category_renaming[x['category']]
    if x['category'] in category_renaming
    else x['category'],
    axis=1
)

# Checando que agora temos as 23 categorias atuais do Oscar
list(oscar_nominations['category'].unique())

In [11]:
# Agora precisamos integrar os esquemas dos datasets do Oscar e de metadados de filmes.

# carregando os metadados de todos os filmes
movies_metadata = pd.read_csv(
    '/kaggle/input/the-movies-dataset/movies_metadata.csv',
)

# converter coluna release_date para datetime
movies_metadata['release_date'] = movies_metadata['release_date'].apply(
    lambda x: pd.to_datetime(x, errors='coerce'),
)

# examinando as características do dataframe
movies_metadata.info()
movies_metadata.describe()
movies_metadata.head()

In [12]:
# Adicionando uma coluna com o ano de lançamento do filme, que será útil em outras análises
movies_metadata['release_year'] = movies_metadata['release_date'].apply(lambda x: x.year)
movies_metadata['release_year'].describe()

In [13]:
# Precisamos retirar os filmes pós 2017 do dataset do oscar, já que não estão presentes no The Movie Dataset
rows_to_drop = oscar_nominations[(oscar_nominations['year_film'] > 2017)].index
oscar_nominations = oscar_nominations.drop(rows_to_drop)

oscar_nominations['year_film'].describe()

In [14]:
# Criando uma função de normalização de nomes para comparação entre os dois datasets
# Essa função será utilizada para evitar que nomes idênticos não sejam encontrados por conta de
# pontuação, capitalização ou caracteres especiais encontrados em exemplos de nomes de filmes
def normalize(name):
    normalized_name = name
    try:
        normalized_name = normalized_name.lower()

        for character in [
            ':', '!', '?', ' -', '- ', '-', '/', '.', '·', ',', '"', '\u200e'
        ]:
            normalized_name = normalized_name.replace(character, '')

        for character_to_replace, replacement in {
            'ž': 'z',
            'ń': 'n',
            '&': 'and',
            '...': ' ',
        }.items():
            if character_to_replace in normalized_name:
                normalized_name = normalized_name.replace(character_to_replace, replacement)

        normalized_name = normalized_name.strip()
                
    except:
        pass

    return normalized_name


movies_metadata['normalized_title'] = movies_metadata['title'].apply(normalize)
movies_metadata['normalized_original_title'] = movies_metadata['original_title'].apply(normalize)
oscar_nominations['normalized_film'] = oscar_nominations['film'].apply(normalize)

In [15]:
# checar filmes do Oscar que não estão no dataset de metadados,
# considerando títulos exatos e dando margem de +- 2 anos de diferença
# entre as datas existentes nas tabelas
not_found_movies = []
found_movies = []

total_nominations = len(oscar_nominations)

i = 0
for nomination in oscar_nominations.iterrows():
    i += 1
    print(f'Analysing nomination {i} of {total_nominations}', end='\r')
    
    data = nomination[1]
    normalized_film = data['normalized_film']
    year_film = data['year_film']
    
    found_title = None
    
    # Checando nos títulos em inglês
    if len(
        movies_metadata.loc[
            (movies_metadata['normalized_title'] == normalized_film) & 
            (movies_metadata['release_year'] >= year_film - 2) &
            (movies_metadata['release_year'] <= year_film + 2)
        ]
    ):
        found_title = normalized_film

    # Checando nos títulos originais
    elif len(
        movies_metadata.loc[
            (movies_metadata['normalized_original_title'] == normalized_film) & 
            (movies_metadata['release_year'] >= year_film - 2) &
            (movies_metadata['release_year'] <= year_film + 2)
        ]
    ):
        # separar título em inglês
        found_title = movies_metadata.loc[
            (movies_metadata['normalized_original_title'] == normalized_film) & 
            (movies_metadata['release_year'] >= year_film - 2) &
            (movies_metadata['release_year'] <= year_film + 2)
        ]['normalized_title'].values[0]
    
    # Se não encontrado, considerar caso especial: filmes com os dois títulos no nome,
    # separados por parênteses, "'s" ou outros caracteres especiais
    else:
        titles = [
            title.strip() for title in
            normalized_film.replace("'s", '*****').replace('(', '*****').replace(')', '*****').split('*****')
            if title
        ]
        
        for title in titles:
            # título parcial encontrado na coluna normalized_title
            if len(
                movies_metadata.loc[
                    (movies_metadata['normalized_title'] == title) & 
                    (movies_metadata['release_year'] >= year_film - 2) &
                    (movies_metadata['release_year'] <= year_film + 2)
                ]
            ):
                found_title = title
            
            # título parcial encontrado na coluna normalized_original_title
            elif len(
                movies_metadata.loc[
                    (movies_metadata['normalized_original_title'] == title) & 
                    (movies_metadata['release_year'] >= year_film - 2) &
                    (movies_metadata['release_year'] <= year_film + 2)
                ]
            ):
                found_title = movies_metadata.loc[
                    (movies_metadata['normalized_original_title'] == title) & 
                    (movies_metadata['release_year'] >= year_film - 2) &
                    (movies_metadata['release_year'] <= year_film + 2)
                ]['normalized_title'].values[0]
                
            
    if found_title:
        found_movies.append((found_title, year_film))

        found_year = movies_metadata.loc[
            (movies_metadata['normalized_title'] == found_title) & 
            (movies_metadata['release_year'] >= year_film - 2) &
            (movies_metadata['release_year'] <= year_film + 2)
        ]['release_year'].values[0]
        
        # se o título encontrado é diferente do presente no dataset dos Oscars, atualizar o dataset
        if found_title != normalized_film:            
            oscar_nominations.loc[
                (oscar_nominations['normalized_film'] == normalized_film) & 
                (oscar_nominations['year_film'] == year_film),
                'normalized_film'
            ] = found_title
        
        # se o ano encontrado é diferente do presente no dataset dos Oscars, atualizar o dataset
        if found_year != year_film:            
            oscar_nominations.loc[
                (oscar_nominations['normalized_film'] == normalized_film) & 
                (oscar_nominations['year_film'] == year_film),
                'year_film'
            ] = found_year

    else:
        not_found_movies.append((normalized_film, year_film))
    
        
found_movies = list(set(found_movies))
print('\n\nFound', len(found_movies), 'movies')

not_found_movies = list(set(not_found_movies))
print('Did not find', len(not_found_movies), 'movies')

In [16]:
# Checar caso a caso filmes que podem estar com nomes levemente diferentes,
# (ou com anos de lançamento diferentes) em cada um dos dois datasets,
# realizando uma análise de similaridade entre os títulos normalizados
# e considerando até dois anos de margem de erro para mais ou para menos

def similarity(a, b):
    try:
        return SequenceMatcher(None, a, b).ratio()
    except:
        return 0


i = 0
total = len(not_found_movies)
possibly_found = {}

if VERBOSE:
    for oscar_title, year_film in not_found_movies:
        i += 1
        print(f'Analysing not found movie {i} of {total}', end="\r")
        best_match = 0

        for movie in movies_metadata.iterrows():
            data = movie[1]
            normalized_title = data['normalized_title']
            normalized_original_title = data['normalized_original_title']
            release_year = data['release_year']

            # análise 1 - similaridade com o título em inglês
            normalized_title_similarity = similarity(oscar_title, normalized_title)
            if (normalized_title_similarity > best_match) and (release_year - 2 <= year_film <= release_year + 2):
                best_match = normalized_title_similarity
                possibly_found[oscar_title, year_film] = (
                    'normalized_title',
                    normalized_title,
                    release_year,
                    normalized_title_similarity,
                )

            # análise 2 - similaridade com o título original
            normalized_original_title_similarity = similarity(oscar_title, normalized_original_title)
            if (normalized_original_title_similarity > best_match) and (release_year - 2 <= year_film <= release_year + 2):
                best_match = normalized_original_title_similarity
                possibly_found[oscar_title, year_film] = (
                    'normalized_original_title',
                    normalized_original_title,
                    release_year,
                    normalized_original_title_similarity,
                )

    possibly_found
    
    # Salvar possibly_found em arquivo - sem ordenação
    with open('possibly_found', 'w') as file:
        file.write(str(possibly_found))

    possibly_found_formatted = sorted(
        [
            (title_oscar, oscar_year, title_field, title_metadata, metadata_year, title_similarity)
            for (title_oscar, oscar_year), (title_field, title_metadata, metadata_year, title_similarity)
            in possibly_found.items()
        ],
        key=lambda x: x[4]
    )

    print('Not found movies:', possibly_found_formatted, end='\n\n')

    # Salvar possibly_found em arquivo - com ordenação
    with open('possibly_found_formatted', 'w') as file:
        file.write(str(possibly_found_formatted))

In [17]:
# Títulos equivalentes que podem ser reconhecidos nas duas listas a partir do arquivo gerado:
movies_to_rename = {
    ("glen campbelli'll be me", 2014): ('normalized_title', "glen campbell i'll be me", 2014.0, 0.9787234042553191),
    ('mt head', 2002): ('normalized_title', 'mount head', 2002.0, 0.8235294117647058),
    ('the story of the weeping camel', 2004): ('normalized_title', 'the weeping camel', 2003.0, 0.723404255319149),
    ('the lady and the reaper (la dama y la muerte)', 2009): ('normalized_title', 'the lady and the reaper', 2009.0, 0.6764705882352942),
    ('harry potter and the sorcerer\'s stone', 2001): ('normalized_title', 'harry potter and the philosopher\'s stone', 2001.0, 0.8533333333333334),
    ('wardance', 2007): ('normalized_title', 'war dance', 2007.0, 0.9411764705882353),
    ('wallace and gromit in the curse of the wererabbit', 2005): ('normalized_title', 'the curse of the wererabbit', 2005.0, 0.7105263157894737),
    ('the lady in number 6 music saved my life', 2013): ('normalized_title', 'the lady in number 6', 2013.0, 0.6666666666666666),
    ('5 broken cameras', 2012): ('normalized_title', 'five broken cameras', 2011.0, 0.8571428571428571),
    ("precious based on the novel 'push' by sapphire", 2009): ('normalized_title', 'precious', 2009.0, 0.2962962962962963),
    ('birdman or (the unexpected virtue of ignorance)', 2014): ('normalized_title', 'birdman', 2014.0, 0.25925925925925924),
}

# padronizar a tabela do Oscar a partir dos dados da tabela de metadados
for oscar_movie, metadata_movie in movies_to_rename.items():
    oscar_title = oscar_movie[0]
    oscar_year = oscar_movie[1]

    metadata_title = metadata_movie[1]
    metadata_year = metadata_movie[2]
    
    print(f'Renaming movie "{oscar_title}" ({oscar_year}) to "{metadata_title}" ({metadata_year})')
    
    oscar_nominations.loc[
        (oscar_nominations['normalized_film'] == oscar_title) & 
        (oscar_nominations['year_film'] == oscar_year),
        ['normalized_film', 'year_film']
    ] = metadata_title, metadata_year

In [18]:
# Finalmente as duas tabelas estão prontas para serem integradas
# Vamos usar one-hot enconding e utilizar abreviações para os nomes das categorias

CATEGORY_CODES = {
    'ACTOR IN A LEADING ROLE': 'leading_actor',
    'ACTOR IN A SUPPORTING ROLE': 'supporting_actor',
    'ACTRESS IN A LEADING ROLE': 'leading_actress',
    'ACTRESS IN A SUPPORTING ROLE': 'supporting_actress',
    'ANIMATED FEATURE FILM': 'animated_feature',
    'PRODUCTION DESIGN': 'production_design',
    'CINEMATOGRAPHY': 'cinematography',
    'COSTUME DESIGN': 'costume_design',
    'DIRECTING': 'directing',
    'DOCUMENTARY (Feature)': 'documentary_feature',
    'DOCUMENTARY (Short Subject)': 'documentary_short subject',
    'FILM EDITING': 'editing',
    'INTERNATIONAL FEATURE FILM': 'international_feature',
    'MAKEUP AND HAIRSTYLING': 'makeup',
    'MUSIC (Original Score)': 'original_score',
    'MUSIC (Original Song)': 'original_song',
    'BEST PICTURE': 'best_picture',
    'SHORT FILM (Animated)': 'animated_short',
    'SHORT FILM (Live Action)': 'live_action_short',
    'SOUND': 'sound',
    'VISUAL EFFECTS': 'visual_effects',
    'WRITING (Adapted Screenplay)': 'writing_adapted',
    'WRITING (Original Screenplay)': 'writing_original',
}

# Criando as colunas para indicação e vitória em cada categoria
for category in oscar_nominations['category'].unique():
    category_code = CATEGORY_CODES[category]
    
    movies_metadata[f'nominated_{category_code}'] = 0
    movies_metadata[f'won_{category_code}'] = 0
    
movies_metadata['year_ceremony'] = None
    
i = 0
found_movies = []
found_nominations = 0
for index, nomination_data in oscar_nominations.iterrows():
    i += 1
    print(f'Analysing nomination {i} of {total_nominations}', end='\r')
    title = nomination_data['normalized_film']
    year_film = nomination_data['year_film']
    year_ceremony = nomination_data['year_ceremony']

    category = nomination_data['category']
    category_code = CATEGORY_CODES[category]

    winner = nomination_data['winner']

    correlated_movies = movies_metadata.loc[
        (movies_metadata['normalized_title'] == title) & 
        (movies_metadata['release_year'] == year_film),
        [f'nominated_{category_code}', f'won_{category_code}', 'year_ceremony']
    ]
    
    if len(correlated_movies) == 1:
        found_nominations += 1
        found_movies.append((title, year_film))
        movies_metadata.loc[
            (movies_metadata['normalized_title'] == title) & 
            (movies_metadata['release_year'] == year_film),
            [f'nominated_{category_code}', f'won_{category_code}', 'year_ceremony']
        ] = 1, int(winner), year_ceremony
        
    elif len(correlated_movies) > 1:
        print(
            movies_metadata[
                (movies_metadata['normalized_title'] == title) & 
                (movies_metadata['release_year'] == year_film)
            ][['title', 'release_year', 'production_countries']]
        )

In [19]:
# teste 1 - validando filmes vencedores do Oscar de melhor filme diretamente na tabela de metadados
movies_metadata[movies_metadata['won_best_picture'] == 1][
    ['title', 'nominated_best_picture', 'year_ceremony']
].values

In [20]:
# teste 2 - validando filmes indicados ao Oscar em 2002
movies_metadata[
    (movies_metadata['nominated_best_picture'] == 1) &
    (movies_metadata['year_ceremony'] == 2002)
][
    ['title', 'nominated_best_picture', 'won_best_picture', 'year_ceremony']
].values

In [21]:
# Não serão interpretadas as colunas a seguir, por não dizerem respeito ao problema em questão,
# ou por não serem quantificáveis para a análise
for column in [
    'belongs_to_collection',
    'homepage',
    'imdb_id',
    'original_title',
    'overview',
    'poster_path',
    'tagline',
]:
    movies_metadata.drop(column, 1, inplace=True)

In [22]:
# checando o número de gêneros existentes, e o número de ocorrências de cada um
all_genres = {}
for genres in movies_metadata[movies_metadata['genres'].notna()]['genres'].values:
    genres = json.loads(genres.replace("'", '"'))

    for genre in genres:
        name = genre['name']
        if name.lower() not in all_genres:
            all_genres[name.lower()] = 0
            
        all_genres[name.lower()] += 1
            
all_genres

In [23]:
# alguns gêneros apresentam apenas uma ocorrência - serão eliminados
valid_genres = []
for genre_name, occurrences in all_genres.items():
    if occurrences > 1:
        valid_genres.append(genre_name.replace(' ', '_'))
        
valid_genres

In [ ]:
# convertendo os códigos de gênero em colunas usando one-hot encoding:
for genre in valid_genres:
    movies_metadata[f'genre_{genre}'] = 0 

total_movies = len(movies_metadata)
i = 0

for movie in movies_metadata.iterrows():
    i += 1
    print(f'Analysing movie {i} of {total_movies}', end='\r')
    index = movie[0]
    genres = movie[1]['genres']
    
    genres = json.loads(genres.replace("'", '"'))
    for genre in genres:
        name = genre['name'].lower().replace(' ', '_')
        if name in valid_genres:
            movies_metadata.loc[
                index,
                f'genre_{name}'
            ] = 1
            
movies_metadata.drop('genres', 1, inplace=True)

In [ ]:
# analisando manualmente filmes de um gênero para conferir
animation_movies = movies_metadata[movies_metadata['genre_animation'] == 1]['title'].values

animation_movies, len(animation_movies)

In [ ]:
# checando o número de países de produção existentes, e o número de ocorrências de cada um
all_countries = {}
for countries in movies_metadata[movies_metadata['production_countries'].notna()]['production_countries'].values:
    countries = json.loads(countries.replace("D'I", 'DI').replace("People's", "Peoples").replace("'", '"'))

    if isinstance(countries, list):
        for country in countries:
            code = country['iso_3166_1']
            if code not in all_countries:
                all_countries[code] = 0

            all_countries[code] += 1
        
    
print(f'{len(all_countries)} countries found')

# convertendo os países de produção em colunas usando one-hot encoding:
for country in all_countries:
    movies_metadata[f'country_{country}'] = 0 

total_movies = len(movies_metadata)
i = 0

for movie in movies_metadata.iterrows():
    i += 1
    print(f'Adding country columns to movie {i} of {total_movies}', end='\r')
    index = movie[0]
    countries = movie[1]['production_countries']
    
    if isinstance(countries, str):
        countries = json.loads(countries.replace("D'I", 'DI').replace("People's", "Peoples").replace("'", '"'))
        if isinstance(countries, list):
            for country in countries:
                code = country['iso_3166_1']
                movies_metadata.loc[
                    index,
                    f'country_{code}'
                ] = 1
            
movies_metadata.drop('production_countries', 1, inplace=True)

In [ ]:
# Analisando a quantidade de empresas produtoras
all_companies = {}
i = 0
for companies in movies_metadata[movies_metadata['production_companies'].notna()]['production_companies'].values:
    i += 1
    print(f'Analysing movie {i} of {total_movies}. Number of companies: {len(all_companies)}', end='\r')

    try:
        # evitando caracteres problema em nomes estrangeiros
        companies = companies.replace("'", '"')
        companies = json.loads(companies)
        for company in companies:
            name = company['name']
            if name.lower() not in all_companies:
                all_companies[name.lower()] = 0

            all_companies[name.lower()] += 1
            
    except Exception as e:
        pass

# mesmo ignorando nomes problemáticos de produtoras, temos mais de 22 mil empresas listadas.
# a análise desse item poderia trazer informações relevantes, mas seria extremamente problemática

# removendo a coluna 
movies_metadata.drop('production_companies', 1, inplace=True)

In [ ]:
# Conferindo o esquema após essas alterações
if VERBOSE:
    movies_metadata.info(verbose=True)

In [ ]:
# calculando as correlações entre as colunas
corrs = movies_metadata.corr().abs()
sorted_corrs = corrs.unstack().sort_values(kind="quicksort", ascending=False)

In [ ]:
# testando um threshold arbitrário, para imprimir apenas as maiores correlações
CORR_THRESHOLD = 0.4

large_corrs = {}
for (a, b), row in sorted_corrs.iteritems():
    if a != b and row > CORR_THRESHOLD and (a, b) not in large_corrs and (b, a) not in large_corrs:
        large_corrs[(a,b)] = row
        
len(large_corrs), large_corrs